In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

In [2]:
# 1. Load & index
df = pd.read_csv(
    'origin_with_indicators.csv',
    parse_dates=['time']
)
df.sort_values('time', inplace=True)
df.set_index('time', inplace=True)

In [3]:
# 2. Train/test split (last 7 days as test)
cutoff = df.index.max() - pd.Timedelta(days=7)
train_df = df[df.index <= cutoff]
test_df  = df[df.index  > cutoff]

In [4]:
# 3. Define which columns to scale vs encode
numeric_cols = [
    'candlebody', 'candleupperwick', 'candlelowerwick',
    'tick_volume',
    'CPI_Actual', 'CPI_Forecast', 'CPI_Previous',
    'GDP_Actual', 'GDP_Forecast', 'GDP_Previous',
    'Interest_Rate_Actual', 'Interest_Rate_Forecast', 'Interest_Rate_Previous',
    'PCE_Actual', 'PCE_Forecast', 'PCE_Previous',
    'PPI_Actual', 'PPI_Forecast', 'PPI_Previous',
    'Macro_factor', 'RSI', 'MACD', 'Signal',
    'SMA_50', 'SMA_200', 'SMA_280'
]
categorical_cols = ['candletype']

In [5]:
# 4. ColumnTransformer: standardize numerics, one‑hot encode candletype
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

preprocessor = ColumnTransformer(transformers=[
    # numeric pipeline stays the same
    ('num', StandardScaler(), numeric_cols),

    # categorical pipeline: use sparse_output and include the cols in the tuple
    ('cat',
     OneHotEncoder(sparse_output=False, handle_unknown='ignore'),
     categorical_cols),
])


In [6]:
# 5. Fit on train, transform train & test
X_train = preprocessor.fit_transform(
    train_df[numeric_cols + categorical_cols]
)
X_test  = preprocessor.transform(
    test_df[numeric_cols + categorical_cols]
)

In [7]:
# --- after fitting & transforming ---
# Build column names: numeric + one‑hot
ohe = preprocessor.named_transformers_['cat']
ohe_cols = list(ohe.get_feature_names_out(categorical_cols))
all_feature_names = numeric_cols + ohe_cols

# Create a DataFrame so we can .head()
X_train_df = pd.DataFrame(
    X_train,
    index=train_df.index,
    columns=all_feature_names
)

print("Head of normalized & encoded training features:")
print(X_train_df.head())


Head of normalized & encoded training features:
                     candlebody  candleupperwick  candlelowerwick  \
time                                                                
2019-12-06 03:30:00   -0.691528        -0.166775        -0.270148   
2019-12-06 04:00:00   -0.687030        -0.535228        -0.467790   
2019-12-06 04:30:00   -0.678035        -0.307654        -0.191091   
2019-12-06 05:00:00   -0.381183        -0.535228        -0.467790   
2019-12-06 05:30:00   -0.516116        -0.405186        -0.270148   

                     tick_volume  CPI_Actual  CPI_Forecast  CPI_Previous  \
time                                                                       
2019-12-06 03:30:00    -0.885440   -0.949621     -0.979522     -0.972283   
2019-12-06 04:00:00    -1.016080   -0.949621     -0.979522     -0.972283   
2019-12-06 04:30:00    -0.946153   -0.949621     -0.979522     -0.972283   
2019-12-06 05:00:00    -1.052941   -0.949621     -0.979522     -0.972283   
2019-12-06 0

In [8]:
# 6. Scale price target separately
price_scaler = StandardScaler()
y_train_price = price_scaler.fit_transform(
    train_df[['price']].values
)
y_test_price  = price_scaler.transform(
    test_df[['price']].values
)

In [9]:
# 7. Sequence builder
SEQ_LEN = 48  # use past 48 bars (~1 trading day) to predict next step
def make_sequences(X, y=None, seq_len=SEQ_LEN):
    Xs, ys = [], []
    for i in range(len(X) - seq_len):
        Xs.append(X[i:i+seq_len])
        if y is not None:
            ys.append(y[i+seq_len])
    return np.array(Xs), (np.array(ys) if y is not None else None)

In [10]:
# 8. Build sequences for feature model
Xf_train, _        = make_sequences(X_train,   y=None)
Xf_test,  _        = make_sequences(X_test,    y=None)

In [11]:
# 9. Build sequences for price model
Xp_train, yp_train = make_sequences(X_train,   y_train_price)
Xp_test,  yp_test  = make_sequences(X_test,    y_test_price)

print("Sequence shapes:")
print("  Xf_train:", Xf_train.shape)
print("  Xf_test: ", Xf_test.shape)
print("  Xp_train:", Xp_train.shape, " yp_train:", yp_train.shape)
print("  Xp_test: ", Xp_test.shape,  " yp_test: ",  yp_test.shape)

Sequence shapes:
  Xf_train: (63825, 48, 28)
  Xf_test:  (182, 48, 28)
  Xp_train: (63825, 48, 28)  yp_train: (63825, 1)
  Xp_test:  (182, 48, 28)  yp_test:  (182, 1)


In [12]:
# 10. Feature‑forecast LSTM
n_feats = Xf_train.shape[2]
feat_model = Sequential([
    LSTM(64, input_shape=(SEQ_LEN, n_feats), return_sequences=True),
    Dense(n_feats)
])
feat_model.compile(optimizer='adam', loss='mse')
es1 = EarlyStopping(monitor='val_loss', patience=5,
                    restore_best_weights=True)
feat_model.fit(
    Xf_train, Xf_train,  # predicting next‑step features uses transformed train itself
    validation_split=0.1,
    epochs=50, batch_size=32,
    callbacks=[es1],
    verbose=1
)

c:\Users\WW\Desktop\FYP\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
1796/1796 ━━━━━━━━━━━━━━━━━━━━ 33s 17ms/step - loss: 0.1325 - val_loss: 0.0478
Epoch 2/50
1796/1796 ━━━━━━━━━━━━━━━━━━━━ 28s 15ms/step - loss: 0.0061 - val_loss: 0.0213
Epoch 3/50
1796/1796 ━━━━━━━━━━━━━━━━━━━━ 27s 15ms/step - loss: 0.0036 - val_loss: 0.0126
Epoch 4/50
1796/1796 ━━━━━━━━━━━━━━━━━━━━ 31s 17ms/step - loss: 0.0022 - val_loss: 0.0100
Epoch 5/50
1796/1796 ━━━━━━━━━━━━━━━━━━━━ 38s 21ms/step - loss: 0.0015 - val_loss: 0.0082
Epoch 6/50
1796/1796 ━━━━━━━━━━━━━━━━━━━━ 41s 23ms/step - loss: 0.0012 - val_loss: 0.0084
Epoch 7/50
1796/1796 ━━━━━━━━━━━━━━━━━━━━ 45s 25ms/step - loss: 0.0011 - val_loss: 0.0082
Epoch 8/50
1796/1796 ━━━━━━━━━━━━━━━━━━━━ 34s 19ms/step - loss: 0.0010 - val_loss: 0.0072
Epoch 9/50
1796/1796 ━━━━━━━━━━━━━━━━━━━━ 30s 16ms/step - loss: 6.6374e-04 - val_loss: 0.0063
Epoch 10/50
1796/1796 ━━━━━━━━━━━━━━━━━━━━ 32s 18ms/step - loss: 5.9250e-04 - val_loss: 0.0065
Epoch 11/50
1796/1796 ━━━━━━━━━━━━━━━━━━━━ 33s 18ms/step - loss: 4.4432e-04 - val_loss: 0.0

In [13]:
# 11. Recursive feature forecast (next 7 days)
pred_feats_scaled = []
last_seq = X_train[-SEQ_LEN:].copy()

for _ in range(len(test_df)):
    # predict one step
    nxt = feat_model.predict(last_seq[np.newaxis])[0]
    pred_feats_scaled.append(nxt)
    # roll the window forward (drop the oldest row, append the new one)
    last_seq = np.vstack([last_seq[1:], nxt])

pred_feats_scaled = np.array(pred_feats_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 442ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 317ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 533ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step   
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 14s 14s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 26s 26s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 55s 55s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 108s 108s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 239s 239s/step


ResourceExhaustedError: Graph execution error:

Detected at node sequential_1/lstm_1/TensorArrayUnstack/TensorListFromTensor defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "c:\Users\WW\Desktop\FYP\venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "c:\Users\WW\Desktop\FYP\venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "c:\Users\WW\Desktop\FYP\venv\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "c:\Users\WW\Desktop\FYP\venv\Lib\site-packages\tornado\platform\asyncio.py", line 211, in start

  File "C:\Users\WW\AppData\Local\Programs\Python\Python312\Lib\asyncio\base_events.py", line 641, in run_forever

  File "C:\Users\WW\AppData\Local\Programs\Python\Python312\Lib\asyncio\base_events.py", line 1987, in _run_once

  File "C:\Users\WW\AppData\Local\Programs\Python\Python312\Lib\asyncio\event

In [ ]:
# 12. Inverse‑scale to original units
#    Grab the number of numeric inputs directly from the scaler:
num_feat_count = preprocessor.named_transformers_['num'].scale_.shape[0]

#    Slice off just those columns:
pred_feats_num = pred_feats_scaled[:, :num_feat_count]

#    And inverse‑transform:
pred_feats_num = preprocessor.named_transformers_['num'].inverse_transform(pred_feats_num)

NameError: name 'preprocessor' is not defined

In [ ]:
# 13. Price‑forecast LSTM
price_model = Sequential([
    LSTM(64, input_shape=(SEQ_LEN, n_feats)),
    Dense(1)
])
price_model.compile(optimizer='adam', loss='mse')
es2 = EarlyStopping(monitor='val_loss', patience=5,
                    restore_best_weights=True)
price_model.fit(
    Xp_train, yp_train,
    validation_split=0.1,
    epochs=50, batch_size=32,
    callbacks=[es2],
    verbose=1
)

In [ ]:
# 14. Recursive price forecast
pred_price_scaled = []
for i in range(len(test_df)):
    if i < SEQ_LEN:
        seq_in = np.vstack([
            X_train[-(SEQ_LEN-i):],
            pred_feats_scaled[:i]
        ])
    else:
        seq_in = pred_feats_scaled[i-SEQ_LEN:i]
    ps = price_model.predict(seq_in[np.newaxis])[0]
    pred_price_scaled.append(ps)

pred_price_scaled = np.array(pred_price_scaled)
pred_price = price_scaler.inverse_transform(pred_price_scaled)

In [ ]:
# 15. Compute & print per‑feature accuracy (RMSE & MAE)
print("\nFeature errors (numeric only):")
print("Feature           \tRMSE\tMAE")
print("--------------------------------------")
for idx, col in enumerate(numeric_cols):
    true = test_df[col].values
    pred = pred_feats_num[:, idx]
    rmse = np.sqrt(mean_squared_error(true, pred))
    mae  = mean_absolute_error(true, pred)
    print(f"{col:15s}\t{rmse:.4f}\t{mae:.4f}")

In [ ]:
# 16. Price RMSE
price_rmse = np.sqrt(mean_squared_error(
    test_df['price'], pred_price[:,0]
))
print(f"\nPrice RMSE over next week: {price_rmse:.4f}")

In [ ]:
# 17. (Optional) View predicted features as DataFrame
pred_feat_df = pd.DataFrame(
    pred_feats_num,
    index=test_df.index,
    columns=numeric_cols
)
print("\nFirst 10 rows of predicted numeric features:")
print(pred_feat_df.head(10))

In [ ]:
# 1. Build a DataFrame of your predicted features
pred_feat_df = pd.DataFrame(
    pred_feats,                     # NumPy array of predictions
    index=test_df.index,            # timestamps for the next 7 days
    columns=FEATURE_COLS            # feature names
)

# 2. Print the entire table
pd.set_option('display.max_rows', None)   # if you really want every 336 rows (7 days × 48 bars)
print(pred_feat_df)

# (Optionally, to just peek at the first few:)
print("\nFirst 10 predicted rows:")
print(pred_feat_df.head(10))

                     candletype  candlebody  candleupperwick  candlelowerwick  \
time                                                                            
2025-05-05 05:00:00    0.470598    2.113744         0.557843         0.623181   
2025-05-05 05:30:00    0.468011    2.098599         0.592646         0.529395   
2025-05-05 06:00:00    0.470989    2.151275         0.621053         0.463787   
2025-05-05 06:30:00    0.472470    2.238684         0.653173         0.453647   
2025-05-05 07:00:00    0.473342    2.295794         0.671349         0.443339   
2025-05-05 07:30:00    0.473316    2.302307         0.673401         0.430217   
2025-05-05 08:00:00    0.472879    2.278402         0.665981         0.419220   
2025-05-05 08:30:00    0.472636    2.254551         0.657916         0.413124   
2025-05-05 09:00:00    0.473048    2.250583         0.654482         0.411452   
2025-05-05 09:30:00    0.474110    2.266912         0.655795         0.411555   
2025-05-05 10:00:00    0.475

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Compute and print per‑feature accuracy
metrics = {}
for idx, col in enumerate(FEATURE_COLS):
    true_vals = test_df[col].values
    pred_vals = pred_feats[:, idx]
    rmse = np.sqrt(mean_squared_error(true_vals, pred_vals))
    mae  = mean_absolute_error(true_vals, pred_vals)
    metrics[col] = {'RMSE': rmse, 'MAE': mae}

print("Feature        \tRMSE\tMAE")
print("---------------------------------")
for feat, vals in metrics.items():
    print(f"{feat:12s}\t{vals['RMSE']:.4f}\t{vals['MAE']:.4f}")

Feature        	RMSE	MAE
---------------------------------
candletype  	0.4978	0.4962
candlebody  	6.6829	4.0602
candleupperwick	2.0595	1.3641
candlelowerwick	1.8315	1.1656
tick_volume 	1901.6706	1522.1438
CPI_Actual  	0.0209	0.0181
CPI_Forecast	0.0141	0.0124
CPI_Previous	0.0062	0.0053
GDP_Actual  	0.2399	0.2141
GDP_Forecast	0.2510	0.2227
GDP_Previous	0.3405	0.2947
Interest_Rate_Actual	0.0329	0.0262
Interest_Rate_Forecast	0.0323	0.0258
Interest_Rate_Previous	0.0308	0.0240
PCE_Actual  	1.9594	1.6431
PCE_Forecast	0.7865	0.6177
PCE_Previous	2.2350	1.8737
PPI_Actual  	0.0085	0.0075
PPI_Forecast	0.0079	0.0068
PPI_Previous	0.0162	0.0135
Macro_factor	493.2169	472.8918
RSI         	27.3369	23.1638
MACD        	18.7447	15.3904
Signal      	17.6109	14.4670
SMA_50      	946.1738	870.1812
SMA_200     	864.0305	776.7498
SMA_280     	830.5221	748.7522


In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

# --- after you have pred_feats and test_df ready ---

metrics = {}
for idx, col in enumerate(FEATURE_COLS):
    true = test_df[col].values
    pred = pred_feats[:, idx]
    rmse = np.sqrt(mean_squared_error(true, pred))
    mae  = mean_absolute_error(true, pred)
    # avoid div/0 by masking zeros
    nonzero = true != 0
    mape = np.mean(np.abs((true[nonzero] - pred[nonzero]) / true[nonzero])) * 100
    pct_acc = 100.0 - mape

    metrics[col] = {
        'RMSE': rmse,
        'MAE': mae,
        'MAPE(%)': mape,
        'Accuracy(%)': pct_acc
    }

#  print a nice table
print(f"{'Feature':20s}\tRMSE\tMAE\tMAPE(%)\tAcc(%)")
print("-"*60)
for feat, v in metrics.items():
    print(f"{feat:20s}\t"
          f"{v['RMSE']:.4f}\t"
          f"{v['MAE']:.4f}\t"
          f"{v['MAPE(%)']:.2f}\t"
          f"{v['Accuracy(%)']:.2f}")


Feature             	RMSE	MAE	MAPE(%)	Acc(%)
------------------------------------------------------------
candletype          	0.4978	0.4962	51.87	48.13
candlebody          	6.6829	4.0602	164.79	-64.79
candleupperwick     	2.0595	1.3641	107.03	-7.03
candlelowerwick     	1.8315	1.1656	245.24	-145.24
tick_volume         	1901.6706	1522.1438	38.30	61.70
CPI_Actual          	0.0209	0.0181	75.54	24.46
CPI_Forecast        	0.0141	0.0124	49.80	50.20
CPI_Previous        	0.0062	0.0053	19.06	80.94
GDP_Actual          	0.2399	0.2141	7138.03	-7038.03
GDP_Forecast        	0.2510	0.2227	11137.38	-11037.38
GDP_Previous        	0.3405	0.2947	1227.91	-1127.91
Interest_Rate_Actual	0.0329	0.0262	58.26	41.74
Interest_Rate_Forecast	0.0323	0.0258	57.33	42.67
Interest_Rate_Previous	0.0308	0.0240	53.28	46.72
PCE_Actual          	1.9594	1.6431	7144.08	-7044.08
PCE_Forecast        	0.7865	0.6177	2807.70	-2707.70
PCE_Previous        	2.2350	1.8737	6939.52	-6839.52
PPI_Actual          	0.0085	0.0075	27.78	72.22
